In [18]:
## import module and dependelcy
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [19]:
# Define url vars
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [20]:
# Get data request
data = requests.get(standings_url)
# data.text

In [21]:
# Pars out  html
soup = BeautifulSoup(data.text, 'html.parser')
# soup


In [22]:
# filter out table from html
standings_table = soup.find_all('table')[0]
#standings_table

In [23]:
# find object in table
links = standings_table.find_all('a')

In [24]:
links = [l.get("href") for l in links]
# links

In [25]:
# filter out url that av squad in the links
links = [l for l in links if '/squads/' in l]
# links

In [26]:
# format sting to full url
team_url = [f"https://fbref.com/{l}" for l in links]
# team_url https://fbref.com//en/squads/18bb7c10/Arsenal-Stats',

In [27]:
# Get team detail URL and detail information

In [28]:
team_url = team_url[0]


In [29]:
data = requests.get(team_url)
# data.text

In [30]:
# Get list of tetail infroamtion on teams
matches = pd.read_html(data.text, match="Scores & Fixtures ")

In [31]:
# List first team details

# matches[0]

In [32]:
# Get shouting stat from speci team

In [33]:
# Pars out  html and find the link that contain "all_comps/shooting/"
soup = BeautifulSoup(data.text, 'html.parser')
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
links

['/en/squads/18bb7c10/2022-2023/matchlogs/all_comps/shooting/Arsenal-Match-Logs-All-Competitions',
 '/en/squads/18bb7c10/2022-2023/matchlogs/all_comps/shooting/Arsenal-Match-Logs-All-Competitions',
 '/en/squads/18bb7c10/2022-2023/matchlogs/all_comps/shooting/Arsenal-Match-Logs-All-Competitions',
 '/en/squads/18bb7c10/2022-2023/matchlogs/all_comps/shooting/Arsenal-Match-Logs-All-Competitions']

In [34]:
# Get team shooting stat from define url
data = requests.get(f"https://fbref.com/{links[0]}")

In [35]:
# Get shooting infromation
shooting = pd.read_html(data.text, match="Shooting")[0]


In [36]:
# Remove first index rowes
shooting.columns = shooting.columns.droplevel()
# shootin"g.head()
# shooting["Date"]

In [37]:
# Combine shooting datafram with match datafram

In [38]:
team_data = matches[0].merge(shooting[["Date","Sh","SoT","Dist"]], on = "Date")
team_data.shape


(21, 22)

In [39]:
# team_data.head()

In [40]:
# Full funkation to scrape the webb.
# function for downlod infromation from multiple teams and years

In [41]:
## import module and dependelcy
import requests
import pandas as pd
import time

from bs4 import BeautifulSoup

# static vars
years = list(range(2022, 2020, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

print(years)

[2022, 2021]


In [ ]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.find_all('table')[0]
    
    
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    revious_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com/{revious_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures ")
        
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
            
        data = requests.get(f"https://fbref.com/{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        shooting.head()

        try:
            team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

        except ValueError:
            continue
        
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["team"] =  team_name
        all_matches.append(team_data)        
        time.sleep(2)



In [45]:
len(all_matches)

40

In [44]:
matche_df = pd.concat(all_matches)
matche_df.columns = [c.lower() for c in matche_df.columns] 
matche_df.to_csv("matches.csv")

In [ ]:
## check file

In [49]:
matches = pd.read_csv("matches.csv", index_col=0)
#matches.head
matches.shape

(1325, 27)

In [ ]:
#filet = (matches["Comp"] == "Premier League")
#matches[filet].value_counts
matches["team"].value_counts()